# Python has a Built-in Database – Here’s How to use it

Source code from Better Data Science ["Python has a Built-in Database – Here’s How to use it"](https://www.betterdatascience.com/python-has-a-built-in-database-heres-how-to-use-it/) written by [Dario Radečić](https://www.betterdatascience.com/author/betterdatascience_onouc8/).

# Importing libraries

In [1]:
import pandas as pd

import sqlite3 as lite

# Functions

In [2]:
def table_exists(conn, table_name):
    
    c.execute('''SELECT
                   COUNT(name)
                 FROM
                   sqlite_master
                 WHERE
                 TYPE = 'table' and
                 name = '{}' '''.format(table_name))
    
    if c.fetchone()[0] == 1:
        
        return True
    
    return False

def insert_movie(conn, c, movie_id, name, release_year, genre, rating):
    
    c.execute('''
              INSERT INTO
                movies (movie_id, name, release_year, genre, rating)
                VALUES (?, ?, ?, ?, ?)
              ''', (movie_id, name, release_year,genre, rating))
    
    conn.commit()
    
def get_movies(c):
    
    c.execute('''SELECT * FROM movies ''')
    
    data = []
    
    for row in c.fetchall():
        
        data.append(row)
        
    return data

def get_movie(c, movie_id):
    
    c.execute('''
              SELECT *
              FROM
                movies
              WHERE
                movie_id = {}
              '''.format(movie_id))
    
    data = []
    
    for row in c.fetchall():
        
        data.append(row)
        
    return data

def update_movie(conn, c, movie_id, update_dict):
    
    valid_keys = ['name', 'release_year', 'genre', 'rating']
    
    for key in update_dict.keys():
        
        if key not in valid_keys:
            
            raise Exception('Invalid field name!')
            
        for key in update_dict.keys():
            
            if type(update_dict[key] == str):
                
                stmt = '''
                       UPDATE movies
                       SET
                         {} = '{}'
                       WHERE
                         movie_id = {}
                       '''.format(key, update_dict[key], movie_id)
                
            else:
                
                stmt = '''
                       UPDATE movies
                       SET
                         {} = {}
                       WHERE
                         movie_id = {}
                       '''.format(key, update_dict[key], movie_id)
                
            c.execute(stmt)
            
        conn.commit()
        
def delete_movie(conn, c, movie_id):
    
    c.execute('''DELETE FROM movies WHERE movie_id = {}'''.format(movie_id))
    conn.commit()            

# Creating tables

In [16]:
conn = lite.connect('movies')

c = conn.cursor()

if not table_exists(conn, 'movies'):
    
    c.execute('''
              CREATE TABLE movies (
                  movie_id INTEGER,
                  name TEXT,
                  release_year INTEGER,
                  genre TEXT,
                  rating REAL
              )
              '''
             )            

# Test time

In [6]:
insert_movie(conn, c, 1, 'Titanic', 1997, 'Drama', 7.8)
insert_movie(conn, c, 2, 'The Day After Tomorrow', 2004, 'action', 6.4)
insert_movie(conn, c, 3, '2012', 2009, 'Action', 5.8)
insert_movie(conn, c, 4, 'Men in Black', 1997, 'Action', 7.3)
insert_movie(conn, c, 5, 'World War Z', 2013, 'Romance', 10)

In [7]:
print(get_movies(c))

[(1, 'Titanic', 1997, 'Drama', 7.8), (2, 'The Day After Tomorrow', 2004, 'action', 6.4), (3, '2012', 2009, 'Action', 5.8), (4, 'Men in Black', 1997, 'Action', 7.3), (5, 'World War Z', 2013, 'Romance', 10.0)]


In [9]:
print(get_movie(c, 2))

[(2, 'The Day After Tomorrow', 2004, 'action', 6.4)]


In [11]:
update_movie(conn, c, 5, {'genre': 'Horror', 'rating': 7.0})

In [12]:
print(get_movie(c, 5))

[(5, 'World War Z', 2013, 'Horror', 7.0)]


In [13]:
delete_movie(conn, c, 3)

In [14]:
print(get_movies(c))

[(1, 'Titanic', 1997, 'Drama', 7.8), (2, 'The Day After Tomorrow', 2004, 'action', 6.4), (4, 'Men in Black', 1997, 'Action', 7.3), (5, 'World War Z', 2013, 'Horror', 7.0)]
